In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Word Embeddings: Encoding Lexical Semantics
===========================================

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn\'t say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where $V$ is
our vocabulary, but often the outputs are only a few dimensional (if we
are only predicting a handful of labels, for instance). How do we get
from a massive dimensional space to a smaller dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

$$\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}$$

where the 1 is in a location unique to $w$. Any other word will have a 1
in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let\'s see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

-   The mathematician ran to the store.
-   The physicist ran to the store.
-   The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

-   The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn\'t it be
much better if we could use the following two facts:

-   We have seen mathematician and physicist in the same role in a
    sentence. Somehow they have a semantic relation.
-   We have seen mathematician in the same role in this new unseen
    sentence as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven\'t. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the [distributional
hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics).

Getting Dense Word Embeddings
-----------------------------

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the \"is able to
run\" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

$$q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
\overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]$$

$$q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
\overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]$$

Then we can get a measure of similarity between these words by doing:

$$\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}$$

Although it is more common to normalize by the lengths:

$$\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
{\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)$$

Where $\phi$ is the angle between the two vectors. That way, extremely
similar words (words whose embeddings point in the same direction) will
have similarity 1. Extremely dissimilar words should have similarity -1.

You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.

In summary, **word embeddings are a representation of the \*semantics\*
of a word, efficiently encoding semantic information that might be
relevant to the task at hand**. You can embed other things too: part of
speech tags, parse trees, anything! The idea of feature embeddings is
central to the field.

Word Embeddings in Pytorch
--------------------------

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$ is the
dimensionality of the embeddings, such that the word assigned index $i$
has its embedding stored in the $i$\'th row of the matrix. In all of my
code, the mapping from words to indices is a dictionary named
word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).


In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


An Example: N-Gram Language Modeling
====================================

Recall that in an n-gram language model, given a sequence of words $w$,
we want to compute

$$P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )$$

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.


In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
[521.625584602356, 519.0713486671448, 516.5341033935547, 514.0120182037354, 511.5031621456146, 509.007116317749, 506.5243458747864, 504.054993391037, 501.5971927642822, 499.1482117176056]
tensor([-1.1796, -1.4428, -0.7074,  0.7528, -0.0203,  1.1036,  0.5368,  0.9796,
        -1.1941, -0.4913], grad_fn=<SelectBackward0>)


Exercise: Computing Word Embeddings: Continuous Bag-of-Words
============================================================

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typically, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an $N$
context window on each side, $w_{i-1}, \dots, w_{i-N}$ and
$w_{i+1}, \dots, w_{i+N}$, referring to all context words collectively
as $C$, CBOW tries to minimize

$$-\log p(w_i | C) = -\log \text{Softmax}\left(A(\sum_{w \in C} q_w) + b\right)$$

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

-   Think about which parameters you need to define.
-   Make sure you know what shape each operation expects. Use .view() if
    you need to reshape.


In [9]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".lower().split()

print(raw_text)

['we', 'are', 'about', 'to', 'study', 'the', 'idea', 'of', 'a', 'computational', 'process.', 'computational', 'processes', 'are', 'abstract', 'beings', 'that', 'inhabit', 'computers.', 'as', 'they', 'evolve,', 'processes', 'manipulate', 'other', 'abstract', 'things', 'called', 'data.', 'the', 'evolution', 'of', 'a', 'process', 'is', 'directed', 'by', 'a', 'pattern', 'of', 'rules', 'called', 'a', 'program.', 'people', 'create', 'programs', 'to', 'direct', 'processes.', 'in', 'effect,', 'we', 'conjure', 'the', 'spirits', 'of', 'the', 'computer', 'with', 'our', 'spells.']


In [11]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".lower().split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


print(vocab)
print(vocab_size)

class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# Create your model and train. Here are some functions to help you make
# the data ready for use by your module.


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['are', 'we', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]
{'of', 'in', 'are', 'we', 'idea', 'rules', 'processes', 'beings', 'manipulate', 'direct', 'evolution', 'effect,', 'evolve,', 'conjure', 'by', 'to', 'about', 'that', 'program.', 'computational', 'people', 'programs', 'create', 'spells.', 'inhabit', 'a', 'is', 'things', 'processes.', 'computers.', 'as', 'process', 'computer', 'other', 'with', 'they', 'data.', 'abstract', 'the', 'pattern', 'study', 'process.', 'directed', 'spirits', 'called', 'our'}
46


tensor([ 2,  3, 15, 40])

In [14]:
print(vocab)

{'of', 'in', 'are', 'we', 'idea', 'rules', 'processes', 'beings', 'manipulate', 'direct', 'evolution', 'effect,', 'evolve,', 'conjure', 'by', 'to', 'about', 'that', 'program.', 'computational', 'people', 'programs', 'create', 'spells.', 'inhabit', 'a', 'is', 'things', 'processes.', 'computers.', 'as', 'process', 'computer', 'other', 'with', 'they', 'data.', 'abstract', 'the', 'pattern', 'study', 'process.', 'directed', 'spirits', 'called', 'our'}


In [15]:
print(vocab_size)

46


In [17]:
#Max iters

max_iters=10

for e in range (max_iters):
  for x, t in data:
    print("......")
    print(x)
    print(t)

......
['are', 'we', 'to', 'study']
about
......
['about', 'are', 'study', 'the']
to
......
['to', 'about', 'the', 'idea']
study
......
['study', 'to', 'idea', 'of']
the
......
['the', 'study', 'of', 'a']
idea
......
['idea', 'the', 'a', 'computational']
of
......
['of', 'idea', 'computational', 'process.']
a
......
['a', 'of', 'process.', 'computational']
computational
......
['computational', 'a', 'computational', 'processes']
process.
......
['process.', 'computational', 'processes', 'are']
computational
......
['computational', 'process.', 'are', 'abstract']
processes
......
['processes', 'computational', 'abstract', 'beings']
are
......
['are', 'processes', 'beings', 'that']
abstract
......
['abstract', 'are', 'that', 'inhabit']
beings
......
['beings', 'abstract', 'inhabit', 'computers.']
that
......
['that', 'beings', 'computers.', 'as']
inhabit
......
['inhabit', 'that', 'as', 'they']
computers.
......
['computers.', 'inhabit', 'they', 'evolve,']
as
......
['as', 'computers.', 

In [18]:
import numpy as np

In [28]:
#Matriz de embeddings
embedding_dim = 10

np.random.random((vocab_size, embedding_dim)).astype(np.float32)

vocab_size = len(vocab)
embeddings = nn.Embedding(vocab_size, embedding_dim)

print(embeddings)

E = torch.tensor(np.random.random((vocab_size, embedding_dim)).astype(np.float32))
print(E.shape)



Embedding(46, 10)
torch.Size([46, 10])


In [27]:
#Asociar palabra a fila que quieres
word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

word_to_ix['with']

[(['are', 'we', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


34

In [32]:
E[word_to_ix['with']].shape

torch.Size([10])

In [40]:
max_iters=1

for e in range (max_iters):
  for x, t in data:
    e_list=[]
    for p in x:
      ix=[p]
      ix.append(word_to_ix[p])
      e_list.append(E[word_to_ix[p],:])
    print(ix)
    print(e_list)
    print(t)

    print("......")

['study', 40]
[tensor([0.2954, 0.8656, 0.3396, 0.5681, 0.7535, 0.4349, 0.4316, 0.4111, 0.0571,
        0.5633]), tensor([0.0255, 0.9877, 0.1933, 0.9089, 0.9616, 0.1103, 0.7102, 0.6352, 0.8589,
        0.9440]), tensor([0.7541, 0.6077, 0.2650, 0.5333, 0.9792, 0.6850, 0.0662, 0.9781, 0.2187,
        0.0377]), tensor([0.2922, 0.9507, 0.8412, 0.7651, 0.3008, 0.6587, 0.9570, 0.1500, 0.1978,
        0.5724])]
about
......
['the', 38]
[tensor([3.5641e-01, 6.5804e-01, 8.7291e-01, 1.9301e-01, 7.1936e-01, 7.5361e-04,
        7.1932e-01, 8.6467e-02, 9.5575e-01, 2.9149e-02]), tensor([0.2954, 0.8656, 0.3396, 0.5681, 0.7535, 0.4349, 0.4316, 0.4111, 0.0571,
        0.5633]), tensor([0.2922, 0.9507, 0.8412, 0.7651, 0.3008, 0.6587, 0.9570, 0.1500, 0.1978,
        0.5724]), tensor([0.7978, 0.2689, 0.6447, 0.3266, 0.1243, 0.6557, 0.3232, 0.1399, 0.9302,
        0.7345])]
to
......
['idea', 4]
[tensor([0.7541, 0.6077, 0.2650, 0.5333, 0.9792, 0.6850, 0.0662, 0.9781, 0.2187,
        0.0377]), tensor([3.5641

In [10]:
#NUEVO TEXTO

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """Ice skating is the self-propulsion and gliding of a person across an ice surface, using metal-bladed ice skates.
People skate for various reasons, including recreation, exercise, competitive sports, and commuting.
Ice skating may be performed on naturally frozen bodies of water, such as ponds, lakes, canals, and rivers,
and on human-made ice surfaces both indoors and outdoors.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# Create your model and train. Here are some functions to help you make
# the data ready for use by your module.


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['skating', 'Ice', 'the', 'self-propulsion'], 'is'), (['is', 'skating', 'self-propulsion', 'and'], 'the'), (['the', 'is', 'and', 'gliding'], 'self-propulsion'), (['self-propulsion', 'the', 'gliding', 'of'], 'and'), (['and', 'self-propulsion', 'of', 'a'], 'gliding')]


tensor([26, 37, 39, 43])